This notebook reproduces the nonlinear Pendulum results from [Champion et. al](https://www.pnas.org/doi/full/10.1073/pnas.1906995116). The data generation is specified in the  [appendix](https://www.pnas.org/action/downloadSupplement?doi=10.1073%2Fpnas.1906995116&file=pnas.1906995116.sapp.pdf) of [Champion et. al](https://www.pnas.org/doi/full/10.1073/pnas.1906995116), and is restated here. 






The nonlinear pendulum equation is given by

\begin{equation}
\ddot{z} = -sin(z)
\end{equation}

Here $z$ denotes the angle between the vertical and the pendulum

As per the appendix "We generate synthetic video of the pendulum in two spatial dimensions by creating high-dimensional snapshots given by"

$$
x(y_1, y_2, t) = exp \left( -20 \left(  (y_1 - cos(z(t)) - \pi/2)\right)^2 + (y_2  - sin(z(t)) - \pi/2)^2 \right)
$$


"at a discretization of $y_1, y_2 \in [−1.5, 1.5]$ . We use 51 grid points in each dimension resulting in snapshots $x(t) \in \mathbb{R}^{2601}$. To generate
a training set, we simulate Eq. (1) from 100 randomly chosen initial conditions with $z(0) \in [-\pi, \pi]$ and $\dot{z(0)} \in [−2.1, 2.1]$. The
156 initial conditions are selected from a uniform distribution in the specified range but are restricted to conditions for which the
pendulum does not have enough energy to do a full loop. This condition is determined by checking that $|\dot{z}(0)^2/2−cos z(0)| ≤ 0.99$."